# Ejercicio 1 - Control de calidad con dos modos de inspección

Una instalación de producción produce **un artículo por hora**. Cada artículo es **defectuoso con probabilidad $p$**, y la calidad de los artículos sucesivos es **independiente**. Se considera la siguiente política de control de calidad parametrizada por dos enteros positivos $k$ y $r$:

* Al inicio, la política exige **inspección del 100%** (modo **costoso**).
* **Tan pronto como** se encuentren **$k$** artículos **consecutivos no defectuosos**, se cambia al **modo económico** y se ordena **inspeccionar cada artículo con probabilidad $1/r$**.
* Se **vuelve al modo costoso** tan pronto como se detecte que **un artículo inspeccionado es defectuoso**.
* El proceso **alterna** de esta manera **para siempre**.

## **(a)** Modela la política de inspección **como una cadena de Markov**, indicando variables, **espacio de estados** y la **matriz de transición** (o **generadora**, según corresponda) **en forma implícita**.

### Parametros
- Hay $k$ articulos en total
- Empezamos en **modo costoso** donde la inspeccionar de subir siempre sera 1
- Cuando estamos en $k$ estamos en **modo costoso**, donde ahora inspeccionar sera $\frac{1}{r}$
- Probabilidad de encontrar un defectuoso es $p$, y de no defectuoso $1-p$
- Nos salimos de **modo economico** apenas detectemos un defectuoso.
### Variable de estado
$$
X_{n}=\text{Numero de articulos no defectuosos consecutivos en modo costoso cada hora }n
$$
### Espacio de estados
$$
S_{x}=\{0,1,\dots,k\}
$$
donde:
- $X_{t}=j(\text{con }0\leq j\leq k-1):$ **modo costoso con racha de $j$ buenas consecutivas** 
- $X_{t}=k:$ **modo economico.**
### Matriz de probabilidades

$$
\begin{bmatrix}
&0&1&2&3&4&5 \\
0&p&1-p&0&0&0&0 \\
1&p&0&1-p&0&0&0 \\
2&p&0&0&1-p&0&0 \\
3 \\
4 \\
5&\frac{1}{r}*p&0&0&0&0&\left( 1-\frac{1}{r} \right)+\left( \frac{1}{r}*(1-p) \right)
\end{bmatrix}
$$
$$
P_{i,j}\rightarrow\begin{cases}
p&j=0 \land i<k \\
1-p&j=i+1 \land i<k \\
\frac{p}{r}&i=k \land j=5 \\
1-\frac{p}{r}&i=k\land j=k \\
0&d.l.c
\end{cases}
$$


In [2]:
import numpy as np
from jmarkov.dtmc import dtmc

In [6]:

# Parametros

k = 20 # Numero de estados
r = 4 # Constante para probabilidad
p = 0.8 # Probabilidad de articulo defectuoso

# Definir espacio de estado

estados_consecutivos = np.array([i for i in range(0,k+1)])
estados_consecutivos

# Creo una matriz de transicion de ceros
matrizConsecutivos = np.zeros((len(estados_consecutivos),len(estados_consecutivos)))
matrizConsecutivos  

# Recorro sobre el estado inicial
for i in range(len(estados_consecutivos)):
    # Recorro sobre el estado futuro:
    for j in range(len(estados_consecutivos)):
        if j == 0 and i < k:
            matrizConsecutivos[i][j] = p
        elif j == i + 1 and i < k:
            matrizConsecutivos[i][j] = 1 - p
        elif i == k and j == 0:
            matrizConsecutivos[i][j] = (p/r)
        elif i == k and j == k:
            matrizConsecutivos[i][j] = ((1/r)*(1 - p))+(1-(1/r))
        else:
            matrizConsecutivos[i][j] = 0

In [7]:
cadenaConsecutivos = dtmc(matrizConsecutivos, estados_consecutivos)

# Ejercicio 2 - Hogar de paso: alimentación y descanso

## Enunciado
La Secretaría Distrital de Integración Social (SDIS) es la entidad del Distrito encargada de liderar y formular las políticas sociales del Distrito Capital para la integración social de las personas, las familias y las comunidades, brindando especial atención a aquellas que están en mayor situación de pobreza y vulnerabilidad. Uno dek los servicios sociales que brinda la SDIS es el **hogar de paso día** para los habitantes de calle. En este lugar, los usuarios tienen la posibilidad de **alimentarse, bañarse, descansar**, entre otras actividades.

El equipo técnico de los servicios sociales de la SDIS ha determinado que la **llegada de personas** al hogar de paso día sigue un **Proceso de Poisson** $(\lambda\ \text{horas}^{-1})$. Cuando una persona llega al hogar se hace un **pequeño registro** en la recepción, y posteriormente pasa al **área de alimentación**. Se estima que el **tiempo de comer** sigue una **Exponencial** $(\mu\ \text{horas}^{-1})$. También se sabe que el **$p\%$** de las personas **abandona** el hogar **después de comer**. Las personas restantes, después de comer, se dirigen al **área de descanso** donde pueden dormir y bañarse. Se estima que el **tiempo de descanso** sigue una **Exponencial** $(\gamma\ \text{horas}^{-1})$. **Después de descansar, un tercio** de las personas **regresa a alimentación** y las demás **salen** del lugar.

Tenga en cuenta que el **hogar de paso día tiene capacidad para $N$ personas**. Suponga que **todos los otros tiempos** del flujo son **despreciables** y que en la **recepción siempre hay espacio** disponible para atender a las personas que llegan.

## **(a)** Formule un **modelo** que permita a la SDIS analizar las **dinámicas de ocupación** del área de **alimentación** y de **descanso** a lo largo del tiempo. **Enuncie todos los supuestos** necesarios y **defina claramente** todos los **componentes del modelo**. Plantee, de forma general, las **tasas o probabilidades de transición** entre estados, según corresponda.

### Diagrama

![[Drawing 2025-08-26 00.40.51.excalidraw]]

### Parámetros
- Las llegadas siguen un **Proceso de Poisson** (exponencial) con tasa $\lambda$

### Variable de estado

$$
X_{t}=(a,d) \text{ donde }a\text{ es el numero de personas en Alimentacion y }d\text{ es el numero de personas en Descanso en el instante }t 
$$
### Espacio de estados
$$
S_{x}=\{(a,d)\in \mathbb{Z}_{\geq_{0}}^{2}:a+d\leq N\}
$$
### Matriz generativa
N = 2
$$
\begin{bmatrix}
&(0,0)&(1,0)&(0,1)&(2,0)&(1,1)&(0,2) \\
(0,0)&\text{Diagonal}&\lambda&0&0&0&0 \\
(1,0)&p*\mu*1&\text{Diagonal}&(1-p)*\mu*1&\lambda&0&0 \\
(0,1)&\frac{2}{3}*\gamma*1&\frac{1}{3}*\gamma*1&\text{Diagonal}&0&\lambda&0 \\
(2,0)&0&p*\mu*2&0&\text{Diagonal}&(1-p)*\mu*2&0 \\
(1,1)&0&\frac{2}{3}*\gamma*1&p*\mu*1&\frac{1}{3}*\gamma*1&\text{Diagonal}&(1-p)*\mu*1 \\
(0,2)&0&0&\frac{2}{3}*\gamma*2&0&\frac{1}{3}*\gamma*2&\text{Diagonal} \\
\end{bmatrix}
$$

$$
P_{(a,d)\rightarrow(a',d')}=\begin{cases}
\lambda&(a',d')=(a+1,d) \\
p\mu a&(a',d')=(a-1,d) \\
(1-p)\mu a&(a',d')=(a-1,d+1) \\
\frac{1}{3}\gamma d&(a',d')=(a+1,d-1) \\
\frac{2}{3}\gamma d&(a',d')=(a,d-1) \\
0&d.l.c
\end{cases}
$$


In [1]:
import numpy as np
from jmarkov.ctmc import ctmc

In [2]:
# Parametros

N = 5
p = 0.3
lambda_llegadas = 15
mu_alimentacion = 7
gamma_descanso = 2

In [3]:
estadosAlimentacion = np.array([i for i in range(0,N+1)])
estadosDescanso = np.array([i for i in range(0,N+1)])

estados_conjunto = []
for i in estadosAlimentacion:
    for j in estadosDescanso:
        if i + j <= N:
            estados_conjunto.append((i,j))
            
estados_conjunto = np.array(estados_conjunto)

In [6]:
matrizSDIS = np.zeros((len(estados_conjunto), len(estados_conjunto)))

for fila, estado_inicial in enumerate(estados_conjunto):
    a,d  = estado_inicial
    
    for columna, estado_futuro in enumerate(estados_conjunto):
        a_, d_ = estado_futuro

        if a_ == a+1 and d_ == d:
            matrizSDIS[fila][columna] = lambda_llegadas
        elif a_ == a-1 and d_ == d:
            matrizSDIS[fila][columna] =p *mu_alimentacion*a
        elif a_ == a-1 and d_ == d+1:
            matrizSDIS[fila][columna] =  1-p *mu_alimentacion*a
        elif a_ == a+1 and d_ == d-1:
         matrizSDIS[fila][columna] =   (1/3)*gamma_descanso*d
        elif a_ == a and d_ == d-1:
         matrizSDIS[fila][columna] =   (2/3)*gamma_descanso*d
        else:
           matrizSDIS[fila][columna] = 0 
    # Asignar la diagonal como el negativo de la suma de las tasas de salida
    matrizSDIS[fila][fila] = -np.sum(matrizSDIS[fila])

In [7]:
cadena = ctmc(matrizSDIS, estados_conjunto)